In [0]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
import random
import shutil
import time

from google.colab import drive
from sklearn.model_selection import train_test_split

In [0]:
!pip install ipython-autotime
%load_ext autotime

In [0]:
%tensorflow_version 2.x
import tensorflow as tf

time: 1.55 s


In [0]:
# Check GPU
!nvidia-smi

Wed Apr  1 15:36:22 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P8    32W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

### Download and preprocess the starter CSV

In [0]:
# Download the text file
def download_if_missing(url, target, extract=True):
  if os.path.exists(target):
    return target
  return tf.keras.utils.get_file(target, origin=url, extract=extract)

colab_root = "/content/"
csv_path = os.path.join(colab_root, "starter2.csv")
download_if_missing("https://storage.googleapis.com/applied-dl/mini-vqa/starter2.csv",
                     csv_path)

'/content/starter2.csv'

time: 16.1 ms


In [0]:
# Process the text file
# Load the text data
import pandas as pd 
import numpy as np

text_data = pd.read_csv("starter2.csv", header=None)
text_data.head()

# Extract different components
questions = text_data[0]
answers = text_data[1]
images_name = text_data[2]

# Convert answers to numeric values
answers_numeric = np.zeros((answers.size,))
answers_numeric[answers=='yes'] = 1
answers_numeric[answers=='no'] = 0
for answer, answer_numeric in zip(answers[:10], answers_numeric):
    print(answer + " " + str(answer_numeric))

yes 1.0
yes 1.0
yes 1.0
yes 1.0
no 0.0
yes 1.0
yes 1.0
yes 1.0
yes 1.0
no 0.0
time: 51.2 ms


## Notebook two: Preprocessing and training
Create a new notebook for this. At the start, download your starter CSV and thumbnails.zip.

The starter code below assumes you have created a zip file called "thumbnails.zip" containing only the resized images from the COOC training set mentioned in the starter CSV, and that you can download it from a URL.

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/
time: 31.1 s


### Convert image names to absolute paths

Next, for convenience, it may be helpful to update the images column in the starter CSV from filenames ```COCO_train2014_000000320111.jpg``` to absolute paths ```/content/images/COCO_train2014_000000320111.jpg```. This will save you some code down the road when it comes time to open them up (you won't need to worry about the relationship between where the starter CSV file is stored, and your images folder).

In [0]:

root_dir = "/content/drive/My Drive"
!unzip -q "/content/drive/My Drive/activation.zip" -d  "/content/"


time: 5min 51s


In [0]:
# # Convert image name to full path and check if all images exist
# image_folder = "/content/drive/My Drive/SelectImage/"
# image_path = []
# image_not_exist = []
# for index, image_name in enumerate(images_name):
#     image_path.append(image_folder + str(image_name))
#     if not os.path.exists(image_path[index]):
#       image_not_exist.append(image_path[index])

# print("Number of non-existent images: " + str(len(image_not_exist)))
# print("Number of loaded images: " + str(len(image_path)))

time: 1.52 ms


In [0]:
# Convert image name to full path and check if all activation exist
activation_folder = "/content/activation/"
image_path = []
image_not_exist = []
for index, image_name in enumerate(images_name):
    image_path.append(activation_folder + str(image_name) + ".npy")
    if not os.path.exists(image_path[index]):
      image_not_exist.append(image_path[index])

print("Number of non-existent images: " + str(len(image_not_exist)))
print("Number of loaded images: " + str(len(image_path)))

Number of non-existent images: 0
Number of loaded images: 20000
time: 4.85 s


In [0]:
# Check a few images and questions
# import numpy as np
# from matplotlib import pyplot as plt

# n_image_check = 20
# index_check = np.random.randint(0, images_name.size, (n_image_check,))
# fig = plt.figure(figsize=(23, 20))
# n_col = 5
# for count, index in enumerate(index_check):
#     # Original
#     original_image = plt.imread(image_path[index])
#     ax = fig.add_subplot(n_image_check/n_col, n_col, count+1)
#     plt.imshow(original_image)
#     ax.set_title(questions[index] + ': ' + answers[index])
#     ax.axes.get_xaxis().set_visible(False)
#     ax.axes.get_yaxis().set_visible(False)  

time: 968 µs


### Shuffle and create a test set
For this assignment, you do not need to download the COCO validation or test sets. Instead, shuffle your starter CSV, and use some of the rows as a test set (say, 4,000 rows). Move them to a separate CSV or data structure, and revisit them later. I realize some of the same images may appear in the train and test set (although with different questions). For this assignment, that's okay. If you prefer, you can write code to ensure the test set has unique images that do not appear in the train set.

In [0]:
# Pick random 4000 samples for test and the rest for training
n_test = 4000
index_shuffle = np.arange(len(image_path))
np.random.shuffle(index_shuffle)
test_image_path = [image_path[index] for index in index_shuffle[:n_test]]
test_answer_numeric = answers_numeric[index_shuffle[:n_test]]
test_questions = list(questions[index_shuffle[:n_test]])

full_train_image_path = [image_path[index] for index in index_shuffle[n_test:]]
full_train_answer_numeric = answers_numeric[index_shuffle[n_test:]]
full_train_questions = list(questions[index_shuffle[n_test:]])

print("Size of test set: " + str(len(test_image_path)) + ",  " + str(test_answer_numeric.shape[0]) + ",  " + str(len(test_questions)))
print("Size of train set: " + str(len(full_train_image_path)) + ",  " + str(full_train_answer_numeric.shape[0]) + ",  " + str(len(full_train_questions)))

Size of test set: 4000,  4000,  4000
Size of train set: 16000,  16000,  16000
time: 32.4 ms


### Create training and validation sets
You may want to do an 80:20 split on your balanced training set, giving you 16,000 training rows, and 4,000 validation rows.

In [0]:
# Your code here
# The scikit-learn utilities (train_test_split) are your friend
n_train = int(len(full_train_image_path) * 0.8)
index_shuffle = np.arange(len(full_train_image_path))
np.random.shuffle(index_shuffle)
train_image_path = [full_train_image_path[index] for index in index_shuffle[:n_train]]
train_answer_numeric = full_train_answer_numeric[index_shuffle[:n_train]]
train_questions = [full_train_questions[index] for index in index_shuffle[:n_train]]

val_image_path = [full_train_image_path[index] for index in index_shuffle[n_train:]]
val_answer_numeric = full_train_answer_numeric[index_shuffle[n_train:]]
val_questions = [full_train_questions[index] for index in index_shuffle[n_train:]]

print("Size of train set: " + str(len(train_image_path)) + ",  " + str(train_answer_numeric.shape[0]) + ",  " + str(len(train_questions)))
print("Size of validation set: " + str(len(val_image_path)) + ",  " + str(val_answer_numeric.shape[0]) + ",  " + str(len(val_questions)))

Size of train set: 12800,  12800,  12800
Size of validation set: 3200,  3200,  3200
time: 17.7 ms


### Verify your train and validation sets look as expected
You cannot spend too much time exploring the data. When I'm developing code for something like this on my own, I often run methods to verify that these splits have the number of rows I expect, to display a few images from each, and to see stats on the class balance. I also write code to verify the splits contain *only* yes/no answers. You may be tired of coding defensively at this point, but I assure you (especially when working with new and increasingly complicated datasets, this effort **always** pays off).

In [0]:
# # Check the proportion of split
# print("Proportion of train: " + str(y_train.shape[0] / full_train_answer_numeric.shape[0]))

time: 5.82 ms


In [0]:
# # Check the balance of answers in train and val sets
# p_yes_train = sum(y_train==1) / y_train.shape
# print("Proportion of yes in train sets: " + str(p_yes_train))

# p_yes_val = sum(y_val==1) / y_val.shape
# print("Proportion of yes in train sets: " + str(p_yes_val))

time: 2.99 ms



### Forward your images through InceptionV3, and cache activations to disk

Rather than training a CNN from scratch for your VQA model, you'll begin by using activations from a pretrained model.  

* Instead of forward each image repeatedly through the model while training (which will be slow), let's do that once and save the activations to disk. 

* We're saving them to disk (rather than keeping them in memory) to accomodate different sizes of datasets down the road.

I've written a good deal of this code for you, but you should go through it and carefully understand how it works.

In [0]:
# # Create a feature extraction model.
# # You should not need to modify this (though you may, if you'd like
# # to use a model other than Inception).
# image_model = tf.keras.applications.InceptionV3(include_top=False,
#                                                 weights='imagenet')
# new_input = image_model.input
# hidden_layer = image_model.layers[-1].output
# image_features_extract_model = tf.keras.Model(new_input, hidden_layer)

time: 3.61 ms


In [0]:
# # A method to load an image off disk, and extract activations using 
# # the model above. You should not need to modify this.
# def image_to_activations(image_path):
#   img = tf.io.read_file(image_path)
#   img = tf.image.decode_jpeg(img, channels=3)
#   img = tf.image.resize(img, (299, 299))
#   activations = tf.keras.applications.inception_v3.preprocess_input(img)
#   return activations, image_path

time: 5.06 ms


In [0]:
# # You'll need to extract activations for every image in your train, validation,
# # and test set. First, create a set of the absolute paths to all of these images 
# # (image_path_set). Populate this with the absolute paths to all these images.
# image_path_set = set(image_path)


# print("Images to preprocess", len(image_path_set))
# print("This make take a few minutes")

time: 3.19 ms


In [0]:
# import time
# # This cell will extract activations for each image and save them to disk 
# # in NumPy format. You should not need to modify this.

# # Note: we're not saving these activations to the cloud,
# # but you certainly could if you wanted to skip this step 
# # in the future. If you look into doing that, it's best to save one large
# # zip with the activations, and download that and extract it locally
# # (rather than doing a bunch of network access to retrieve individual files),
# # especially when training your model.
# # Start time stamp
# t = time.time()

# # Create a dataset to load each image off disk, and extract activations
# activation_dataset = tf.data.Dataset.from_tensor_slices(list(image_path_set))
# activation_dataset = activation_dataset.map(
#   image_to_activations, 
#   num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(32)

# # Save all activations to disk in NumPy format
# for img_batch, path_batch in activation_dataset:
#   batch_features = image_features_extract_model(img_batch)
#   for bf, p in zip(batch_features, path_batch):
#     path_of_feature = p.numpy().decode("utf-8")
#     np.save(path_of_feature, bf.numpy())  

# # End time stamp
# elapsed = time.time() - t
# print("Time taken: " + str(elapsed/60) + " minutes")   

time: 2.83 ms


### Create lists of questions, answers, and images for your train, validation, and test set

At this point, you may be reading your starter CSV directly from disk, or you may have your own data structure in memory. Since many of the methods we'll call from this point forward take lists as input, you may find it helpful to create a data structure with three lists, that correspond to the columns from your starter CSV.

In [0]:
# Your code here. Populate these for your training set.
questions_train = list(train_questions) 
answers_train = list(train_answer_numeric) 
images_train = list(train_image_path)

# Your code here. Populate these for your validation set.
questions_val = list(val_questions)
answers_val = list(val_answer_numeric)
images_val = list(val_image_path)

# Your code here. Populate these for your test set
questions_test = list(test_questions)
answers_test = list(test_answer_numeric)
images_test = list(test_image_path)

time: 10.1 ms


In [0]:
# Check any nan values
count_nan = 0
for question in questions_val:
  if question != question:
    count_nan = count_nan + 1

print("Number of nan entries: " + str(count_nan))
for question in questions_train[:10]:
  print(question)

### Create and fit a tokenizer

Your model will use a LSTM to process the questions. First, you'll need to vectorize your text. Tokenize the questions, and limit the vocabulary size to a reasonable size (for example, the top 3,000 words). A larger size will make a higher accuracy possible, but complicate and slow your model. Reminder: fit the tokenizer only on the training questions.

In [0]:
# You should not need to modify this code
VOCAB_SIZE = 3000
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=VOCAB_SIZE)
tokenizer.fit_on_texts(questions_train)

# Note, the tokenizer's word_index will not respect VOCAB_SIZE.
# but, that parameter will be respected in later methods,
# (for example, when you call text_to_sequences).
# Also note that '0' is a reserved index for padding.
print("Number of unique words: ", len(tokenizer.word_index))

Number of unique words:  4953
time: 223 ms


### Vectorize the questions
In this section, you will use your tokenizer to vectorize the questions using ```texts_to_sequences```. For an example of texts_to_sequences, see this [notebook](https://github.com/fchollet/deep-learning-with-python-notebooks/blob/master/6.1-one-hot-encoding-of-words-or-characters.ipynb).


In [0]:
# Use the texts_to_sequences utility to vectorize your training, 
# validation, and test questions. 
sequences_train = tokenizer.texts_to_sequences(questions_train)
sequences_val = tokenizer.texts_to_sequences(questions_val)
sequences_test = tokenizer.texts_to_sequences(questions_test)

time: 271 ms


### Pad sequences

In this section, you will pad the vectorized questions using ```pad_sequences```. Your maximum sequence length is a design decision, just like vocab size. Advice, start with something short, so your model trains faster (maybe, between 10 and 20).

For an example of pad_sequences, see this [notebook](https://github.com/fchollet/deep-learning-with-python-notebooks/blob/master/6.2-understanding-recurrent-neural-networks.ipynb).

In [0]:
# # To choose a reasonable sequence length, examine the length of all the 
# # tokenized questions in the training set (in words).
# # Justify your choice, e.g. you should note that percentage of the 
# # training questions that fall under that length (and will not be trimmed), 
# # and the number of questions that will be.
# token_length_train = []
# for sequence in sequences_train:
#   token_length_train.append(len(sequence))

# plt.figure(figsize=(10, 5))
# n_bin = 50
# plt.hist(token_length_train, n_bin, density=True, histtype='step',
#                            cumulative=True, label='Empirical')
# plt.title("Mean: " + str(sum(token_length_train)/len(token_length_train)))
# plt.grid(True)
# plt.show()

time: 889 µs


In [0]:
# Use the pad_sequences utility to pad your training, 
# validation, and test questions.
MAX_SEQ_LEN = 10 

padded_train = tf.keras.preprocessing.sequence.pad_sequences(sequences_train, maxlen=MAX_SEQ_LEN)
padded_val = tf.keras.preprocessing.sequence.pad_sequences(sequences_val, maxlen=MAX_SEQ_LEN)
padded_test = tf.keras.preprocessing.sequence.pad_sequences(sequences_test, maxlen=MAX_SEQ_LEN)

time: 91.5 ms


### Sanity check
You've just done a **bunch** of preprocessing. Optionally, now would be a good time to write a block of code to verify the tokenized and padded sequences are in the format you expect.

In [0]:
# Check the tokenized set
reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))
def sequence_to_text(list_of_indices):
    # Looking up words in dictionary
    words = [reverse_word_map.get(letter) for letter in list_of_indices]
    return(words)

for sequence, quest in zip(sequences_train[:10], questions_train[:10]):
  print(str(sequence_to_text(sequence)) + " | " + quest)

['is', 'the', 'man', 'looking', 'for', 'food'] | is the man looking for food
['do', 'the', 'elephants', 'want', 'to', 'hurt', 'her'] | do the elephants want to hurt her
['are', 'all', 'these', 'people', 'watching', 'tv'] | are all these people watching tv
['are', 'there', 'many', 'people', 'waiting', 'for', 'the', 'bus'] | are there many people waiting for the bus
['is', 'the', 'bear', 'eating'] | is the bear eating
['is', 'the', 'baby', 'wearing', 'pink'] | is the baby wearing pink
['is', 'there', 'a', 'mountain', 'in', 'the', 'photo'] | is there a mountain in the photo
['does', 'this', 'contain', 'carrots'] | does this contain carrots
['is', 'the', 'giraffe', 'in', 'captivity'] | is the giraffe in captivity
['are', 'the', 'glasses', 'the', 'man', 'is', 'wearing', 'used', 'for', 'reading'] | are the glasses the man is wearing used for reading
time: 10.9 ms


In [0]:
# Check the padded set
reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))

for sequence, quest in zip(padded_train[:10], questions_train[:10]):
  print(str(sequence_to_text(sequence)) + " | " + quest)

[None, None, None, None, 'is', 'the', 'man', 'looking', 'for', 'food'] | is the man looking for food
[None, None, None, 'do', 'the', 'elephants', 'want', 'to', 'hurt', 'her'] | do the elephants want to hurt her
[None, None, None, None, 'are', 'all', 'these', 'people', 'watching', 'tv'] | are all these people watching tv
[None, None, 'are', 'there', 'many', 'people', 'waiting', 'for', 'the', 'bus'] | are there many people waiting for the bus
[None, None, None, None, None, None, 'is', 'the', 'bear', 'eating'] | is the bear eating
[None, None, None, None, None, 'is', 'the', 'baby', 'wearing', 'pink'] | is the baby wearing pink
[None, None, None, 'is', 'there', 'a', 'mountain', 'in', 'the', 'photo'] | is there a mountain in the photo
[None, None, None, None, None, None, 'does', 'this', 'contain', 'carrots'] | does this contain carrots
[None, None, None, None, None, 'is', 'the', 'giraffe', 'in', 'captivity'] | is the giraffe in captivity
['are', 'the', 'glasses', 'the', 'man', 'is', 'wearin

### Create a tf.dataset for training, validation, and testing

The method to create the dataset is provided for you, though you will need to get it working by passing the ```padded_train```, ```answers_train``` ```and images_train``` lists you created above.




In [0]:
BATCH_SIZE = 32
BUFFER_SIZE = 1000

# Load cached activations off disk.
def load_np(img_path, question, answer):
  img_path_str = img_path.decode('utf-8')
  head_tail = os.path.split(img_path_str)
  activation_path = os.path.join("/content", "activation", head_tail[1])
  activations = np.load(activation_path)
  return activations, question, answer, img_path

# You should not need to modify this.

# This method will create a dataset that returns four elements.
# - a batch of activations (loaded from disk)
# - a batch of padded questions
# - a batch of numeric answers
# - a batch of absolute paths to the corresponding images
def create_dataset(images, sequences, answers):
  dataset = tf.data.Dataset.from_tensor_slices((images, 
                                                sequences, 
                                                answers))
  # TODO jbgordon@: rewrite this to be clearer
  # Load the cached activations off disk
  dataset = dataset.map(lambda x, y, z: tf.numpy_function(
      load_np, [x, y, z], [tf.float32, tf.int32, tf.float64, tf.string]),
      num_parallel_calls=tf.data.experimental.AUTOTUNE)
  
  # Shuffle and batch
  dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
  dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
  return dataset

time: 14.5 ms


In [0]:
# Call the above method to create train, val, and test datasets.
# If you want to follow along with the starter code, I suggest these
# variable names:
train_ds = create_dataset(images_train, padded_train, answers_train)
val_ds = create_dataset(images_val, padded_val, answers_val)
test_ds = create_dataset(images_test, padded_test, answers_test)

time: 183 ms


### Sanity check (optional)
That dataset creation method is complicated. Write a block of code that demonstrates how to use the dataset (e.g., retrieve a batch of activations, questions, answeres, and images paths) and verify they look as expected.

In [0]:
# Verify your datasets are working properly

# Here is code you can use to quickly retrieve a batch of data
# my_iterator = iter(train_ds)
# activations_batch, questions_batch, answers_batch, paths_batch = next(my_iterator)


In [0]:
# print(activations_batch.shape,
#       questions_batch.shape, 
#       answers_batch.shape, 
#       paths_batch.shape)


### Define your VQA model

You can base your code off the example give toward bottom of this [guide](https://keras.io/getting-started/functional-api-guide/) (but use a smaller model to start). Aim for a couple hundred thousand parameters or so

In [0]:
from tensorflow.keras.layers import Dense, Embedding, Flatten, Input, LSTM
from tensorflow.keras.models import Model, Sequential

time: 1.04 ms


In [0]:
# Your code here
# Below is starter code for your model for you to complete.
# See https://keras.io/getting-started/functional-api-guide/ for the idea.
# The vision model is written for you. You will need to write the question
# model.

# Input to your vision model (activations from Inception-V3,
# loaded off disk disk by the dataset above).
image_input = Input(shape=(8, 8, 2048)) 
vision_model = Sequential()

# Used to reduce the number of parameters (rather using a dense layer here).
vision_model.add(tf.keras.layers.GlobalAveragePooling2D())
vision_model.add(tf.keras.layers.Flatten())

# Output of your vision model
encoded_image = vision_model(image_input) 

# Your code here
# Write your test processing model that takes the vectorized and padded
# question as input.
# As in the guide above, you will want to produce an `encoded_question`
# as output
embed_dim = 20
question_input = Input(shape=(MAX_SEQ_LEN,), dtype='int32')
embedded_question = Embedding(input_dim=VOCAB_SIZE, output_dim=embed_dim, 
                              input_length=MAX_SEQ_LEN)(question_input)
encoded_question = LSTM(embed_dim)(embedded_question)

# Concatenate the encoded image and question
merged = tf.keras.layers.concatenate([encoded_image, encoded_question])

# Optionally, add a small dense layer
dropout_rate = 0.2
regularize_rate = 0.0001

dense1 = Dense(128, kernel_regularizer=tf.keras.regularizers.l1_l2(regularize_rate))(merged)
batch_norm1 = tf.keras.layers.BatchNormalization()(dense1)
activation1 = tf.keras.layers.Activation("relu")(batch_norm1)
dropout1 = tf.keras.layers.Dropout(dropout_rate)(activation1)

# dense2 = Dense(256)(dropout1)
# batch_norm2 = tf.keras.layers.BatchNormalization()(dense2)
# activation2 = tf.keras.layers.Activation("relu")(batch_norm2)
# dropout2 = tf.keras.layers.Dropout(dropout_rate)(activation2)

# Next, add a binary classifier on top
output = Dense(1, activation='sigmoid')(dropout1)

# Your final model
model = Model(inputs=[image_input, question_input], outputs=output)

model.compile(optimizer='adam', 
              loss='binary_crossentropy',
              metrics=['accuracy'])

time: 622 ms


In [0]:
model.summary()

### Plot your model
Create a schematic that shows the graph of your model, using [plot_model](https://keras.io/visualization/). This can be helpful to ensure the vision and question paths look at expected (it's also super cool, and more informative than .summary).

In [0]:
# You should not need to modify this.
from tensorflow.keras.utils import plot_model
plot_model(model, to_file='model.png')

### Fit your model on a single batch
Before training on your entire dataset, a helpful first step is to train repeatedly on a single batch, and verify the loss goes to zero. If your model is working properly, it should be able to memorize a batch of data. We will use ```model.train_on_batch``` for this.

In [0]:
# You should not nee to modify this.
# Retrieve a batch of data from your train dataset
activations_batch, questions_batch, answers_batch, paths_batch = next(iter(train_ds))

time: 1.01 s


In [0]:
# Train them model repeatedly using model.train_on_batch
# Verify the loss goes to zero after +/- 100 training steps.
# If it does not, now would be a great time to debug 
# before proceeding further.
train_steps = 50
loss = []
accuracy = []
for i in range(train_steps):
  metrics = model.train_on_batch([activations_batch, questions_batch], answers_batch)
  loss.append(metrics[0])
  accuracy.append(metrics[1])


epochs = range(len(accuracy))
fig = plt.figure(figsize=(15, 5))
ax1 = fig.add_subplot(121)
plt.title('Training accuracy')
plt.plot(epochs, accuracy, color='blue', label='Train')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

ax1 = fig.add_subplot(122)
plt.title('Training loss')
plt.plot(epochs, loss, color='blue', label='Train')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

In [0]:
# input("Press Enter to continue...")

### Use your model to make predictions on the same batch above
Compare the predicted answer to the label. Verify they match.

In [0]:
# You should not need to modify this, but you will want to 
# carefully inspect the output.
# for prediction, answer in zip(model.predict(x=[activations_batch, questions_batch]), answers_batch):
#   print(np.round(prediction), answer.numpy())

### Create a checkpoints directory

Once you have been able to train your model on a single batch, it's time to begin training on your training dataset. It may take a while to train for a single epoch, and it would be unfortunate if Colab disconnected after training for a while, and you lost your progress.

* After each training epoch, save the model's weights in a checkpoint file. You can learn more about how to create checkpoints [here](https://www.tensorflow.org/guide/keras/save_and_serialize). 

* To resume training, load the latest checkpoint from disk. This will restore the latest weights and resume your progress. If it does not exist, begin training from scratch.

* As a tip, you may want to store your checkpoints in Google Drive, so you'll still have access to them if Colab disconnects. 

Note that running long jobs is not what Colab is intended for. Normally, you could simple save your checkpoints on the machine you're working on. Saving to drive adds an extra step, but it's worthwhile learning how to do.

Tip: after mounting Drive, **do not** programmatically run any commands to delete files from your checkpoints folder or elsewhere (e.g., by using ```!rm -rf```) in Colab. If you're not careful, you may accidentally wipe out your entire drive if you make a programming mistake. 

* Instead, if you need to delete checkpoints, do so manually through the drive user interface. Note that files deleted through the UI may take a minute or so to "actually" be deleted as reflected by ```!ls``` commands run from Colab.

In [0]:
# If you'd like to save checkpoints in drive, you will need to uncomment
# the code below. Alternatively, you can modify it to save
# checkpoints in Colab (these will not persist if your instance is terminated,
# but you can manually download them if you like).
drive_root = '/content/drive/My Drive/'
checkpoint_dir = os.path.join(drive_root, "checkpoints")
checkpoint_dir = os.path.join(checkpoint_dir, "hw4")

# Used for formatting
checkpoint_path = os.path.join(checkpoint_dir, "cp-{epoch:08d}.ckpt")

time: 3.33 ms


In [0]:
# Uncomment this if you'd like to create a checkpoints folder in your drive
print("Checkpoints directory is", checkpoint_dir)
if os.path.exists(checkpoint_dir):
  print("Checkpoints folder already exists")
else:
  print("Creating a checkpoints directory")
  os.makedirs(checkpoint_dir)

Checkpoints directory is /content/drive/My Drive/checkpoints/hw4
Creating a checkpoints directory
time: 8.8 ms


### Restore the latest checkpoint (if it exists).

In [0]:
# You should not need to modify this code.
latest = tf.train.latest_checkpoint(checkpoint_dir)
if latest != None:
  print("Loading weights from", latest)
  model.load_weights(latest)
else:
  print("Checkpoint not found. Starting from scratch")

Checkpoint not found. Starting from scratch
time: 4.4 ms


### Metrics

In [0]:
# Utilities to help us record metrics.
# You should not need to modify this code
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.Mean(name='train_accuracy')

val_loss = tf.keras.metrics.BinaryCrossentropy(name='val_loss')
val_accuracy = tf.keras.metrics.BinaryAccuracy(name='val_accuracy')

time: 32.6 ms


### Code to evaluate on the validation dataset
The validation dataset may be large. It would be wasteful to evaluate on the entire validation dataset each training epoch. Instead, you could evaluate every N epochs, or, you can use the below methoid to evaluate for a fixed number of steps (batches). This will give you a noisier evaluation, but a useful indicator of how your model is doing over time.

In [0]:
# You should not need to modify this.
def evaluate(max_steps=None):
  steps = 0
  for activation_batch, question_batch, answer_batch, path_batch in val_ds:
    if max_steps != None and steps == max_steps:
      break
    predictions = model.predict(x=[activation_batch, question_batch])
    steps += 1 
    # Record metrics after each batch
    val_loss(answer_batch, predictions)
    val_accuracy(answer_batch, predictions)

time: 4.68 ms


### Train the model
As before, we'll write our training loop using ```train_on_batch```. This is intermediate in complexity between using ```.fit``` and writing everything from scratch using a ```GradientTape```. Because we're not using ```.fit```. Since we're not using .fit, there's a bit of extra code we need to write ourselves to track loss and accuracy as we go.

In [0]:
# Used to track loss and accuracy as we go
# You should not need to modify these
MAX_VAL_STEP = 100
train_loss_history, train_acc_history = [], []
val_loss_history, val_acc_history = [], []

last_epoch = -1
epochs = 100 

# Training loop
for epoch in range(last_epoch+1, last_epoch+1+epochs):

  start = time.time()

  # Train for one epoch
  batch = 0
  for activation_batch, question_batch, answer_batch, path_batch in train_ds:
    result = model.train_on_batch(x=[activation_batch, question_batch], y=answer_batch)

    # Record metrics after each batch
    train_loss(result[0])
    train_accuracy(result[1])

  # Evaluate for a few steps
  evaluate(max_steps=MAX_VAL_STEP)

  # Print progress
  # You should not need to modify this.
  template = 'Epoch {}, Loss: {:.2f}, Accuracy: {:.2f}, Val Loss {:.2f}, Val Accuracy {:.2f}, Time: {:.1f} secs'
  print(template.format(epoch,
                        train_loss.result(),
                        train_accuracy.result() * 100,
                        val_loss.result(),
                        val_accuracy.result() * 100,
                        time.time() - start))
  
  # Record history
  train_loss_history.append(train_loss.result())
  train_acc_history.append(train_accuracy.result() * 100)
  val_loss_history.append(val_loss.result())
  val_acc_history.append(val_accuracy.result() * 100)

  # Reset the metrics for the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  val_loss.reset_states()
  val_accuracy.reset_states()

  # Save a checkpoint after each epoch
  model.save_weights(checkpoint_path.format(epoch=epoch), save_format='tf')
  # print("Weights saved")


### Create plots of your training and validation loss and accuracy

In [0]:
# The history object contains results on the training and test
# sets for each epoch
acc = train_acc_history
val_acc = val_acc_history
loss = train_loss_history
val_loss = val_loss_history

# Get the number of epochs
epochs = range(len(acc))
fig = plt.figure(figsize=(15, 5))
ax1 = fig.add_subplot(121)
plt.title('Training and validation accuracy')
plt.plot(epochs, acc, color='blue', label='Train')
plt.plot(epochs, val_acc, color='orange', label='Val')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

ax1 = fig.add_subplot(122)
plt.title('Training and validation loss')
plt.plot(epochs, loss, color='blue', label='Train')
plt.plot(epochs, val_loss, color='orange', label='Val')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

### At this point, you have end-to-end code to train VQA model
Now you can begin working on increasing accuracy. For this assignment, your model should at least be able to fit your training split reasonably well. 

Given the amount of data we're using, you do **not** need to build a model that performs well on validation to receive full credit for this asignment (I recently made the starter dataset a bit more difficult, as a result - you may need to train using more data than I want you to use for a homework assignment, so no worries).

### Writen response
How would you improve the accuracy on the validation set? Please list three ideas below.

#### TODO: your answer here

1. Train certain layers of InceptionV3 network / Make custom model / Use and train on a simpler pre-made model (MobileNet, ...)
2. Data augmentation
3. Get more data

### Finally, evaluate your model on the test set
How well did it do? This assignment is lengthy. For our purposes, it's fine to report the accuracy - and you're done :) 

For a proper evaluation, take a look at this [paper](https://arxiv.org/abs/1612.00837), as discussed in class.

Note: if your model is performing poorly on the test set, you can still receive full credit for this assignment as mentioned above. 

In [0]:
# Restore the latest checkpoint
latest = tf.train.latest_checkpoint(checkpoint_dir)
assert latest != None
model.load_weights(latest)
print("loaded weights from", latest)

In [0]:
# Your code here
# Calculate accuracy on the test set

Phew, that's it! This was a long assignment, I hope it was a useful (and fun!) experience.